In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,73.33,74,62,16.98,broken clouds
1,1,Chokurdakh,RU,70.6333,147.9167,6.19,97,100,2.68,overcast clouds
2,2,Lagoa,PT,39.0500,-27.9833,62.78,68,21,22.39,light rain
3,3,Ribeira Grande,PT,38.5167,-28.7000,61.77,72,88,3.00,overcast clouds
4,4,Maarianhamina,AX,60.0973,19.9348,51.66,100,100,6.91,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Kapaa,US,22.0752,-159.3190,84.52,72,0,17.27,clear sky
13,13,Hilo,US,19.7297,-155.0900,81.25,84,100,8.05,haze
17,17,Atuona,PF,-9.8000,-139.0333,77.49,67,2,18.07,clear sky
18,18,Alofi,NU,-19.0595,-169.9187,78.69,53,75,12.66,broken clouds
30,30,Porbandar,IN,21.6422,69.6093,77.14,66,0,5.70,clear sky
32,32,Kavieng,PG,-2.5744,150.7967,84.22,72,100,6.08,light rain
35,35,Laguna,US,38.4210,-121.4238,76.08,34,0,6.91,clear sky
36,36,Bara,NG,10.3744,10.7288,77.52,30,35,6.46,scattered clouds
48,48,Puerto Escondido,MX,15.8500,-97.0667,85.03,58,75,9.22,broken clouds
55,55,Phangnga,TH,8.4509,98.5298,77.04,93,100,2.55,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                185
City                   185
Country                184
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesnÃ¢Â€Â™t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kapaa,US,84.52,clear sky,22.0752,-159.3190,
13,Hilo,US,81.25,haze,19.7297,-155.0900,
17,Atuona,PF,77.49,clear sky,-9.8000,-139.0333,
18,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,
30,Porbandar,IN,77.14,clear sky,21.6422,69.6093,
32,Kavieng,PG,84.22,light rain,-2.5744,150.7967,
35,Laguna,US,76.08,clear sky,38.4210,-121.4238,
36,Bara,NG,77.52,scattered clouds,10.3744,10.7288,
48,Puerto Escondido,MX,85.03,broken clouds,15.8500,-97.0667,
55,Phangnga,TH,77.04,overcast clouds,8.4509,98.5298,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kapaa,US,84.52,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Hilo,US,81.25,haze,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Atuona,PF,77.49,clear sky,-9.8000,-139.0333,Villa Enata
18,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
30,Porbandar,IN,77.14,clear sky,21.6422,69.6093,Lords Inn Porbandar
32,Kavieng,PG,84.22,light rain,-2.5744,150.7967,Nusa Island Retreat
35,Laguna,US,76.08,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
36,Bara,NG,77.52,scattered clouds,10.3744,10.7288,
48,Puerto Escondido,MX,85.03,broken clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
55,Phangnga,TH,77.04,overcast clouds,8.4509,98.5298,Sunimit Mansion


In [15]:
import numpy as np
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kapaa,US,84.52,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Hilo,US,81.25,haze,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Atuona,PF,77.49,clear sky,-9.8000,-139.0333,Villa Enata
18,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
30,Porbandar,IN,77.14,clear sky,21.6422,69.6093,Lords Inn Porbandar
32,Kavieng,PG,84.22,light rain,-2.5744,150.7967,Nusa Island Retreat
35,Laguna,US,76.08,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
36,Bara,NG,77.52,scattered clouds,10.3744,10.7288,NaN
48,Puerto Escondido,MX,85.03,broken clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
55,Phangnga,TH,77.04,overcast clouds,8.4509,98.5298,Sunimit Mansion


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kapaa,US,84.52,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Hilo,US,81.25,haze,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Atuona,PF,77.49,clear sky,-9.8000,-139.0333,Villa Enata
18,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
30,Porbandar,IN,77.14,clear sky,21.6422,69.6093,Lords Inn Porbandar
32,Kavieng,PG,84.22,light rain,-2.5744,150.7967,Nusa Island Retreat
35,Laguna,US,76.08,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
48,Puerto Escondido,MX,85.03,broken clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
55,Phangnga,TH,77.04,overcast clouds,8.4509,98.5298,Sunimit Mansion
69,Lorengau,PG,85.44,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} Ã‚Â°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))